In [1]:
import numpy as np

## 11.1.1 Carteiras Eficientes com N ativos arriscados

In [2]:
# Matriz de covariancia dos ativos
cov = np.matrix([[0.3, 0.02, 0.01], [0.02, 0.15, 0.03], [0.01, 0.03, 0.18]])

# Retornos esperados dos ativos
ret = np.matrix([[0.08, 0.03, 0.05]]).T

# Retorno esperado da carteira
ret_cart = .08

def fronteira_eficiente(mi, sig, mi_pi):
    N = sig.shape[0]
    ones = np.ones((N,1))
    A = mi.T * np.linalg.inv(sig) * ones
    B = mi.T * np.linalg.inv(sig) * mi
    C = ones.T * np.linalg.inv(sig) * ones
    D = B*C - A**2
    g = (B[0, 0] * np.linalg.inv(sig) * ones - A[0, 0] * np.linalg.inv(sig) * mi) / D[0, 0]
    h = (C[0, 0] * np.linalg.inv(sig) * mi - A[0, 0] * np.linalg.inv(sig) * ones) / D[0, 0]
    W = g + mi_pi * h
    sig_pi = np.sqrt(W.T * sig * W)
    mi_min = -(g.T * sig * h) / (h.T * sig * h)
    sig_min = np.sqrt(g.T * sig * g - ((g.T * sig * h) ** 2) / (h.T * sig   * h))
    return W

## 11.1.2 Carteiras de mercado com N ativos arriscados

In [3]:
# Matriz de covariancia dos ativos
cov = np.matrix([[0.3, 0.02, 0.01], [0.02, 0.15, 0.03], [0.01, 0.03, 0.18]])

# Retornos esperados dos ativos
ret = np.matrix([[0.08, 0.03, 0.05]]).T

# Retorno livre de risco
ret_rf = .02

def carteira_mercado(mi, sig, mi_rf):
    N = sig.shape[0]
    ones = np.ones((N,1))
    lamb = 1 / ((mi - mi_rf).T * np.linalg.inv(sig).T * ones)
    W = (lamb.item(0) * np.linalg.inv(sig) * (mi - mi_rf)).T
    return W

## 11.2.1 Modelo de 1 Fator

In [4]:
# Matriz de retornos dos ativos disponíveis
ret_i = np.matrix(np.random.random((100, 5)))

# Vetor de retornos de um índice de mercado
ret_m = np.matrix(np.random.random((100, 1)))

def modelo_um_fator(ri, rm):
    K = ri.shape[1]
    N = ri.shape[0]
    sig_m = np.std(rm)
    betas = np.zeros((K, 1))
    alphas = np.zeros((K,1))
    e = np.matrix(np.zeros((N,K)))
    for j in range(0, K):
        series = np.vstack((ri[:,j],rm))
        betas[j] = np.cov(series.T)/np.var(rm)
        alphas[j] = np.mean(ri[:,j])-betas[j]*np.mean(rm)
    e[:, j] = ri[:, j] - (alphas.item(j) + betas.item(j) * rm)
    ee = e.T * e * np.diag(np.ones(K))
    omega = sig_m**2 * betas * betas.T + ee
    return omega